# Tutorials

### Blog Agent Tutorial
Here’s how to build a Blog Agent that collects content from a website, generates images, and writes a blog post.

**Step 1: Set API Key**

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "your-api-key"

**Step 2: Import Dependencies**

In [ ]:
from wyge.models.openai import ChatOpenAI
from wyge.tools.prebuilt_tools import extract_relevant_sections_from_website, generate_images_and_add_to_blog

**Step 3: Initialize Tools**

In [ ]:
web_tool = extract_relevant_sections_from_website()  # Tool to scrape a website
blog_tool = generate_images_and_add_to_blog()  # Tool to generate image and add to blog

**Step 4: Initialize LLM with Tools**

In [ ]:
llm = ChatOpenAI(tools=[blog_tool, web_tool], return_tool_output=True, memory=True)

**Step 5: Gather Context**

In [ ]:
topic = "How to tackle AI"
url = "https://www.digiotai.com"

# Collect content from website
prompt1 = (
    f"Gather relevant information about the topic from the website.\n"
    f"Topic: {topic}\n"
    f"Website: {url}"
)
context = llm.run(prompt1)

**Step 6: Generate Blog Post**

In [ ]:
prompt2 = (
    "Write a comprehensive blog post with images for a company based on the following details:\n\n"
    f"Topic: {topic}\n"
    f"Summarized content from company's website: {context}\n\n"
    "The blog should include an engaging introduction to the topic, followed by detailed sections about how the company addresses the topic, "
    "and a conclusion summarizing the key points. Generate '<image> image prompt here </image>' where necessary."
)
blog_content = llm.run(prompt2)

# LinkedIn Agent Tutorial
Create a LinkedIn agent that extracts key takeaways from a YouTube video and posts them on LinkedIn.

**Step 1: Set API Key**

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "your-api-key"

**Step 2: Import Dependencies**

In [ ]:
from vyzeai.models.openai import ChatOpenAI
from vyzeai.tools.prebuilt_tools import youtube_transcript_loader, post_on_linkedin, generate_image_openai

**Step 3: Initialize Tools**

In [ ]:
yt_tool = youtube_transcript_loader()  # Extract YouTube transcript
img_tool = generate_image_openai()  # Generate image
linkedin_tool = post_on_linkedin()  # Post on LinkedIn

**Step 4: Initialize LLM with Tools**

In [ ]:
llm = ChatOpenAI(tools=[yt_tool, img_tool, linkedin_tool], memory=True)

**Step 5: Extract Transcript and Create Post**

In [ ]:
video_url = "https://www.youtube.com/watch?v=e_uOigt1w1o"

prompt1 = (
    "You are an AI agent specialized in creating LinkedIn posts from YouTube videos. "
    "Your task is to extract the key takeaways and insights from the following video and create a professional, "
    "engaging LinkedIn post that summarizes the content."
    f"\nYouTube video URL: {video_url}"
)
linkedin_post = llm.run(prompt1)

# Synthetic Data Generator

In this tutorial, you will learn how to generate synthetic data using VyzeAI's OpenAI integration. We will take a dataset from an Excel file, generate additional synthetic rows that follow the same structure, and append them to the original data.

#### Step 1: Setup Dependencies

Make sure you have the following dependencies installed:

```bash
pip install pandas openai vyzeai

#### Step 2: Load the Data and Initialize the Model
First, load the Excel data and initialize the ChatOpenAI model:

In [ ]:
import pandas as pd
from vyzeai.models.openai import ChatOpenAI

# File path to the Excel file
file_path = "sample_data.xlsx"

# Load the dataset
data = pd.read_excel(file_path)

# Take a sample of the last 50 rows
sample_data = data.tail(50)

# Convert the sample data into a CSV-like string format
sample_str = sample_data.to_csv(index=False, header=False)

# Initialize the OpenAI model
llm = ChatOpenAI(memory=True, api_key="your-api-key-here")

#### Step 3: Generate Synthetic Data
Now, we will generate synthetic data by prompting the model to create rows based on the structure of the existing dataset. The code generates data in chunks to ensure the model can process the request efficiently.

In [ ]:
# Number of rows to generate in each chunk
chunk_size = 50

# Total number of rows to generate
num_rows = 50

# System message to instruct the model to behave as a synthetic data generator
system_message = "You are a synthetic data generator. Provide output in the specified format without extra text."

# Store the generated data
generated_rows = []
rows_generated = 0

# Main loop to generate synthetic data until the desired number of rows is reached
while rows_generated < num_rows:
    # Use previously generated rows or the original sample for generation
    current_sample_str = "\n".join([",".join(row) for row in generated_rows[-50:]]) if generated_rows else sample_str
    
    # Calculate how many rows to generate in this batch
    rows_to_generate = min(chunk_size, num_rows - rows_generated)
    
    # Construct the prompt for the model
    prompt = (f"Generate {rows_to_generate} rows of synthetic data based on this sample:\n\n{current_sample_str}\n"
              "Output as pipe-separated values ('|') without column headers.")

    # Get the generated data from the model
    generated_data = llm.run(prompt, system_message=system_message, return_tool_output=True)
    
    # Parse the generated data into rows
    rows = [row.split("|") for row in generated_data.strip().split("\n") if row]
    
    # Append the new rows to the generated rows
    generated_rows.extend(rows)
    rows_generated += len(rows)

# Create a DataFrame from the generated rows
generated_df = pd.DataFrame(generated_rows, columns=data.columns)

# Combine the original and synthetic data
combined_df = pd.concat([data, generated_df], ignore_index=True)

print(generated_df.head())